# Predicting Hospital Readmission in Diabetic Patients Using Decision Tree Models
## Improving Healthcare Performance Metrics through Enhanced Predictive Modelling and Hyperparameter Optimization
### Pablo X Zumba

This data set for this exercise is from healthcare. It contains data about 10 years of clinical care at 130 US Hospitals. Each row represents a single patient. The columns include the characteristics of deidentified diabetes patients. This is a binary classification task: predict whether a diabetes patient is readmitted to the hospital within 30 days of their discharge (1=Yes, 0=No). This is an important performance metric for hospitals as they try to minimize these types of readmissions.

## Description of Variables

The description of variables are provided in "Healthcare (small) - Data Dictionary.docx"

## Goal

Use the **healthcare_small.csv** data set and build a model to predict **readmitted**. 

# Read and Prepare the Data

In [1]:
# Common imports

import pandas as pd
import numpy as np

np.random.seed(42)

# Get the data

In [2]:
#We will predict the "price" value in the data set:

diabetes = pd.read_csv("healthcare_small.csv")
diabetes

,race,gender,age,admission_type,discharge_disposition,admission_source,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,readmitted
0,Other,Female,70-80,2,3,1,14,32,3,15,0,0,0,9,1
1,Caucasian,Female,80-90,1,3,5,4,44,0,15,0,0,0,9,0
2,AfricanAmerican,Male,50-60,5,1,1,6,29,1,15,0,0,0,9,1
3,Caucasian,Female,50-60,1,1,6,3,47,0,10,0,0,0,4,0
4,AfricanAmerican,Female,40-50,3,1,1,4,92,0,15,0,0,0,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8661,Caucasian,Female,80-90,2,6,7,4,62,0,13,0,0,1,9,1
8662,AfricanAmerican,Female,70-80,1,18,7,6,35,0,12,0,0,1,4,1
8663,Caucasian,Female,50-60,1,3,7,11,41,1,25,2,3,0,9,0
8664,Caucasian,Male,70-80,1,1,7,1,34,0,13,0,0,0,9,1


# Split data (train/test)

In [3]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(diabetes, test_size=0.3)

# Data Prep

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

## Separate the target variable 

In [5]:
train_target = train['readmitted']
test_target = test['readmitted']

train_inputs = train.drop(['readmitted'], axis=1)
test_inputs = test.drop(['readmitted'], axis=1)

## Feature Engineering: Derive a new column

Examples:
- Whether the patient had any emergency visits or not
- Ratio of inpatient visits to outpatient visits
- Ratio of time in hospital to number of medications
- (etc.)

In [6]:
def new_col(df):
    #Create a copy so that we don't overwrite the existing dataframe
    df1 = df.copy()
    
    df1['emergency_binary'] = np.where(df1['number_emergency'] > 0, 1, 0)
    
    return df1[['emergency_binary']]
    # You can use this to check whether the calculation is made correctly:
    #return df1

In [7]:
#Let's test the new function:

# Send train set to the function we created
new_col(train)

,emergency_binary
4166,0
5546,0
2957,0
6329,0
565,0
...,...
5734,0
5191,0
5390,0
860,0


##  Identify the numeric, binary, and categorical columns

In [8]:
# Let's change some of the categorical variables' data type to object

train_inputs['admission_source'] = train_inputs['admission_source'].astype('object')
train_inputs['admission_type'] = train_inputs['admission_type'].astype('object')
train_inputs['discharge_disposition'] = train_inputs['discharge_disposition'].astype('object')

test_inputs['admission_source'] = test_inputs['admission_source'].astype('object')
test_inputs['admission_type'] = test_inputs['admission_type'].astype('object')
test_inputs['discharge_disposition'] = test_inputs['discharge_disposition'].astype('object')


In [9]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [10]:
numeric_columns

['time_in_hospital',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'number_outpatient',
 'number_emergency',
 'number_inpatient',
 'number_diagnoses']

In [11]:
categorical_columns

['race',
 'gender',
 'age',
 'admission_type',
 'discharge_disposition',
 'admission_source']

In [12]:
feat_eng_columns = ['number_emergency']

# Pipeline

In [13]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [14]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [15]:
# Create a pipeline for the transformed column here
my_new_column = Pipeline(steps=[('number_emergency', FunctionTransformer(new_col))])



In [16]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('trans', my_new_column, feat_eng_columns)],   
        remainder='passthrough')

#passtrough is an optional step. You don't have to use it.

# Transform: fit_transform() for TRAIN

In [17]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

<6066x65 sparse matrix of type '<class 'numpy.float64'>'
	with 85723 stored elements in Compressed Sparse Row format>

In [18]:
train_x.shape

(6066, 65)

# Tranform: transform() for TEST

In [19]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

<2600x65 sparse matrix of type '<class 'numpy.float64'>'
	with 36758 stored elements in Compressed Sparse Row format>

In [20]:
test_x.shape

(2600, 65)

# Calculate the Baseline

In [21]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(train_x, train_target)

DummyClassifier(strategy='most_frequent')

In [22]:
from sklearn.metrics import accuracy_score

In [23]:
#Baseline Train Accuracy
dummy_train_pred = dummy_clf.predict(train_x)

baseline_train_acc = accuracy_score(train_target, dummy_train_pred)

print('Baseline Train Accuracy: {}' .format(baseline_train_acc))

Baseline Train Accuracy: 0.5313221233102539


In [24]:
#Baseline Test Accuracy
dummy_test_pred = dummy_clf.predict(test_x)

baseline_test_acc = accuracy_score(test_target, dummy_test_pred)

print('Baseline Test Accuracy: {}' .format(baseline_test_acc))

Baseline Test Accuracy: 0.5365384615384615


# Train a Decision Tree model

In [25]:
#Do NOT train a DecisionTreeClassifier() without any parameters. It OVERFITS. 
from sklearn.tree import DecisionTreeClassifier 

tree_clf = DecisionTreeClassifier(max_depth=7)

tree_clf.fit(train_x, train_target)

DecisionTreeClassifier(max_depth=7)

### Calculate the accuracy

In [26]:
from sklearn.metrics import accuracy_score



In [27]:

#Train accuracy:
train_y_pred = tree_clf.predict(train_x)

print(accuracy_score(train_target, train_y_pred))


0.6391361688097593


In [28]:
#Test accuracy:
test_y_pred = tree_clf.predict(test_x)

print(accuracy_score(test_target, test_y_pred))

0.6038461538461538


# Train another DT model (with different parameters)

In [29]:

#Do NOT train a DecisionTreeClassifier() without any parameters. It OVERFITS. 
from sklearn.tree import DecisionTreeClassifier 

tree_clf = DecisionTreeClassifier(max_depth=6, min_samples_leaf = 17, criterion='entropy')

tree_clf.fit(train_x, train_target)


DecisionTreeClassifier(criterion='entropy', max_depth=6, min_samples_leaf=17)

### Calculate the accuracy

In [30]:

#Train accuracy:
train_y_pred = tree_clf.predict(train_x)

print(accuracy_score(train_target, train_y_pred))


0.6193537751401252


In [31]:
#Test accuracy:
test_y_pred = tree_clf.predict(test_x)

print(accuracy_score(test_target, test_y_pred))

0.5976923076923077


# Optional: try grid search

# Randomized Grid Search

Grid Search helps us try out a combination of different hyperparameters (sometimes randomly)

In [32]:
#Grid Search helps us try out a combination of different hyperparameters (sometimes randomly)
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_grid = {'max_depth': randint(low=7, high=30), 
              'min_samples_leaf': randint(low=10, high=30)}

tree_gs = RandomizedSearchCV(DecisionTreeClassifier(), param_grid, 
                             n_iter=20, cv=5, verbose=1,
                             scoring='accuracy',
                             return_train_score=True)

tree_gs.fit(train_x, train_target)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_iter=20,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fcdcedf8700>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fcdcedfa220>},
                   return_train_score=True, scoring='accuracy', verbose=1)

In [33]:
cvres = tree_gs.cv_results_

for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

0.5865482533400517 {'max_depth': 11, 'min_samples_leaf': 25}
0.5725373527586239 {'max_depth': 15, 'min_samples_leaf': 19}
0.5812743874364892 {'max_depth': 24, 'min_samples_leaf': 26}
0.5700622444115166 {'max_depth': 24, 'min_samples_leaf': 16}
0.5783047735202521 {'max_depth': 10, 'min_samples_leaf': 14}
0.5769880386966566 {'max_depth': 29, 'min_samples_leaf': 25}
0.5863833728783864 {'max_depth': 9, 'min_samples_leaf': 27}
0.5763300108245245 {'max_depth': 29, 'min_samples_leaf': 23}
0.581769028821485 {'max_depth': 21, 'min_samples_leaf': 26}
0.6015533260626572 {'max_depth': 7, 'min_samples_leaf': 26}
0.5707244825755035 {'max_depth': 29, 'min_samples_leaf': 20}
0.5766597717478552 {'max_depth': 23, 'min_samples_leaf': 23}
0.5789664684207738 {'max_depth': 26, 'min_samples_leaf': 28}
0.569732483488186 {'max_depth': 27, 'min_samples_leaf': 16}
0.5774830875292513 {'max_depth': 19, 'min_samples_leaf': 27}
0.5763300108245246 {'max_depth': 29, 'min_samples_leaf': 23}
0.5977609396284892 {'max_dep

In [34]:
#Find the best parameter set
tree_gs.best_params_

{'max_depth': 7, 'min_samples_leaf': 26}

In [35]:
tree_gs.best_estimator_

DecisionTreeClassifier(max_depth=7, min_samples_leaf=26)

In [36]:
#Train accuracy:
train_y_pred = tree_gs.best_estimator_.predict(train_x)

print(accuracy_score(train_target, train_y_pred))

0.6368282228816353


In [37]:
#Test accuracy:
test_y_pred = tree_gs.best_estimator_.predict(test_x)

print(accuracy_score(test_target, test_y_pred))

0.6084615384615385


In [38]:
#Test confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(test_target, test_y_pred)

array([[1022,  373],
       [ 645,  560]])